## Import Individual Stock Returns and Characteristics

In [1]:
# Packages
import numpy as np
import pandas as pd
from pathlib import Path
from pandas.tseries.offsets import MonthEnd
import wrds
#import openassetpricing as oap

# Data directory
datapath = Path('/work/rw196/data/')

# WRDS connection
conn = wrds.Connection(wrds_username='adamwang024')
# conn.describe_table(library='contrib', table='global_factor')

Loading library list...
Done


### From JKP

In [2]:
# # downloading and extracting list of developed countries
# countries = pd.read_excel('https://github.com/bkelly-lab/ReplicationCrisis/raw/master/GlobalFactors/Country%20Classification.xlsx')
# countries_rel = countries[countries['msci_development'] == 'developed']['excntry'].tolist()

# downloading and extracting list of characteristics
chars = pd.read_excel('https://github.com/bkelly-lab/ReplicationCrisis/raw/master/GlobalFactors/Factor%20Details.xlsx')
chars_jkp = chars[chars['abr_jkp'].notna()].sort_values(by=['abr_jkp']).reset_index(drop=True)
chars_jkp = chars_jkp[['abr_jkp','abr_hxz','group','name_new','cite','in-sample period','t-stat','p-value','direction','significance']]
#chars_chosen = chars_jkp[chars_jkp['significance']==1].reset_index(drop=True)
chars_chosen = chars_jkp['abr_jkp'].tolist()
len(chars_chosen)

153

JKP applies 4 screens:
- exch_main=1 (only use data from the main exchanges within a country)
- primary_sec=1 (if a firm has multiple securities outstanding, only retain the primary security as identified by Compustat)
- obs_main=1 (if CRSP and Compustat have data for the same stock, only retain the observation from CRSP)
- common=1 (only retrieve common stocks)

However, these filters may exclude a very small number of SP500 firms. For this reason, I do not use these filters when focusing on S&P 500 firms. 

#### Import the full dataset.

In [3]:
%%time
# Stock characteristics in USA
sql_query= f"""
            SELECT  excntry, eom, permno, size_grp, me, ret, ret_exc, sic, {', '.join(map(str, chars_chosen))}
                    FROM contrib.global_factor
                    WHERE excntry='USA' and eom >= '1959-12-01' and eom <= '2024-12-31' and common=1 and exch_main=1 and primary_sec=1 and obs_main=1
            """

charc = conn.raw_sql(sql_query)

CPU times: user 22min 52s, sys: 43.9 s, total: 23min 35s
Wall time: 28min 9s


In [4]:
charc

,excntry,eom,permno,size_grp,me,ret,ret_exc,sic,age,aliq_at,...,taccruals_at,taccruals_ni,tangibility,tax_gr1a,turnover_126d,turnover_var_126d,z_score,zero_trades_126d,zero_trades_21d,zero_trades_252d
0,USA,1962-02-28,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,26.0,0.73697,...,0.055493,0.691228,0.807338,-0.002817,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
1,USA,1962-01-31,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,1.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,USA,1962-01-31,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,25.0,<NA>,...,<NA>,<NA>,0.743086,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3,USA,1962-01-31,<NA>,mega,372.722384,<NA>,<NA>,<NA>,145.0,0.972656,...,<NA>,<NA>,0.735235,0.011927,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,USA,1962-01-31,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,37.0,<NA>,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
411126,USA,2024-08-31,93436.0,mega,684004.37235,-0.07739,-0.081809,3711.0,236.0,0.84934,...,0.062915,0.504812,0.647934,-0.054016,0.029518,0.360574,9.47273,0.000715,0.000828,0.000661
411127,USA,2024-09-30,93436.0,mega,839047.425659,0.221942,0.217323,3711.0,237.0,0.84934,...,0.062915,0.504812,0.647934,-0.054016,0.028794,0.372432,9.47273,0.000715,0.000826,0.000695
411128,USA,2024-10-31,93436.0,mega,802033.510593,-0.045025,-0.048932,3711.0,238.0,0.851467,...,0.021696,0.197611,0.662048,-0.052716,0.027153,0.382452,10.166609,0.000831,0.001147,0.000753
411129,USA,2024-11-30,93436.0,mega,1107984.321356,0.381469,0.377514,3711.0,239.0,0.851467,...,0.021696,0.197611,0.662048,-0.052716,0.028598,0.41699,10.166609,0.00086,0.000964,0.000804


In [ ]:
# Save
charc.to_parquet(datapath / 'JKP_stock_charcs.parquet', engine='pyarrow')

#### Import SP500 firms only.

In [5]:
# Load SP500 constituents
sp500_cons = pd.read_csv('../Data/constructed/sp500_constituents_monthly.csv')
sp500_cons['date'] = pd.to_datetime(sp500_cons['date'])
sp500_cons

,date,10006,10030,10057,10078,10102,10104,10107,10108,10137,...,92988,93002,93089,93096,93132,93159,93246,93422,93429,93436
0,1959-12-31,0.013055,0.012000,0.021028,NaN,0.083871,NaN,NaN,NaN,-0.048366,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1960-01-31,0.005155,-0.015000,-0.020785,NaN,-0.092262,NaN,NaN,NaN,-0.020833,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1960-02-29,0.046154,0.017767,0.009434,NaN,-0.021312,NaN,NaN,NaN,0.053192,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1960-03-31,-0.059553,-0.100249,-0.047170,NaN,0.038851,NaN,NaN,NaN,-0.005387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1960-04-30,-0.081794,-0.056180,0.000000,NaN,-0.050407,NaN,NaN,NaN,0.010274,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
776,2024-08-31,NaN,NaN,NaN,NaN,NaN,0.013195,-0.001100,NaN,NaN,...,NaN,0.013319,0.042292,-0.310823,0.321675,NaN,0.005460,NaN,0.122718,-0.077391
777,2024-09-30,NaN,NaN,NaN,NaN,NaN,0.206030,0.031548,NaN,NaN,...,NaN,0.062707,-0.016385,0.019284,0.010950,NaN,0.015013,NaN,-0.002580,0.221942
778,2024-10-31,NaN,NaN,NaN,NaN,NaN,-0.012676,-0.055659,NaN,NaN,...,NaN,-0.015826,0.025228,-0.046589,0.014313,NaN,0.041981,NaN,0.042466,-0.045025
779,2024-11-30,NaN,NaN,NaN,NaN,NaN,0.101287,0.044149,NaN,NaN,...,NaN,-0.045297,0.070945,-0.034608,0.208365,NaN,0.136817,NaN,0.013626,0.381469


In [6]:
%%time
# Loop over each month
mdates = sp500_cons['date']
monthly_data = []

for i, dt in enumerate(mdates, 1):
    
    # Get permnos in the S&P500 at this date
    row = sp500_cons[sp500_cons['date'] == dt]
    row_series = row.squeeze()  # convert to Series
    row_series = row_series.drop(labels='date')  # remove date column
    row_series.index = row_series.index.astype(int)
    present_permnos = row_series[row_series.notna()].index

    # Tracker
    print(f"[{i}/{len(mdates)}] Importing data for {dt.strftime('%Y-%m')} with {len(present_permnos)} permnos")

    # Format permnos for SQL
    permno_str = ', '.join(str(int(p)) for p in present_permnos)

    # Run SQL query
    sql_query = f"""
        SELECT excntry, eom, permno, size_grp, me, ret, ret_exc, sic, exch_main, primary_sec, obs_main, common, {', '.join(map(str, chars_chosen))}
        FROM contrib.global_factor
        WHERE excntry = 'USA'
          AND eom = '{dt.strftime('%Y-%m-%d')}'
          AND permno IN ({permno_str})
    """
    df_month = conn.raw_sql(sql_query)
    monthly_data.append(df_month)

# Combine all months into one DataFrame
charc_sp500 = pd.concat(monthly_data, ignore_index=True)

[1/781] Importing data for 1959-12 with 499 permnos
[2/781] Importing data for 1960-01 with 499 permnos
[3/781] Importing data for 1960-02 with 499 permnos
[4/781] Importing data for 1960-03 with 499 permnos
[5/781] Importing data for 1960-04 with 499 permnos
[6/781] Importing data for 1960-05 with 499 permnos
[7/781] Importing data for 1960-06 with 498 permnos
[8/781] Importing data for 1960-07 with 498 permnos
[9/781] Importing data for 1960-08 with 498 permnos
[10/781] Importing data for 1960-09 with 499 permnos
[11/781] Importing data for 1960-10 with 499 permnos
[12/781] Importing data for 1960-11 with 499 permnos
[13/781] Importing data for 1960-12 with 498 permnos
[14/781] Importing data for 1961-01 with 499 permnos
[15/781] Importing data for 1961-02 with 499 permnos
[16/781] Importing data for 1961-03 with 499 permnos
[17/781] Importing data for 1961-04 with 499 permnos
[18/781] Importing data for 1961-05 with 499 permnos
[19/781] Importing data for 1961-06 with 495 permnos
[2

<timed exec>:32: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.


CPU times: user 1min 24s, sys: 2.92 s, total: 1min 27s
Wall time: 4min 56s


In [7]:
charc_sp500

,excntry,eom,permno,size_grp,me,ret,ret_exc,sic,exch_main,primary_sec,...,taccruals_at,taccruals_ni,tangibility,tax_gr1a,turnover_126d,turnover_var_126d,z_score,zero_trades_126d,zero_trades_21d,zero_trades_252d
0,USA,1959-12-31,10006.0,large,68.773,0.013055,0.01053,3740.0,1.0,1.0,...,0.020185,1.974854,0.820941,-0.042259,0.001111,0.727633,<NA>,0.002782,0.003799,0.002735
1,USA,1959-12-31,10030.0,large,81.2,0.012,0.009475,3310.0,1.0,1.0,...,<NA>,<NA>,<NA>,<NA>,0.000878,0.728669,<NA>,0.003604,0.004186,0.003916
2,USA,1959-12-31,10057.0,small,27.0625,0.021028,0.018503,3540.0,1.0,1.0,...,<NA>,<NA>,<NA>,<NA>,0.00052,0.763964,<NA>,1.493707,0.961344,1.831166
3,USA,1959-12-31,10102.0,mega,328.356,0.083871,0.081346,2810.0,1.0,1.0,...,0.063869,0.976929,0.834817,-0.013663,0.000414,0.581053,<NA>,0.006423,0.006311,0.006781
4,USA,1959-12-31,10137.0,mega,323.136,-0.048366,-0.050891,4910.0,1.0,1.0,...,<NA>,<NA>,<NA>,0.000452,0.000191,0.73445,<NA>,0.009002,0.009126,0.008753
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
390643,USA,2024-12-31,93096.0,large,16674.789253,-0.018765,-0.022428,5331.0,1.0,1.0,...,-0.065393,-1.469655,<NA>,-0.007652,0.01871,1.009771,2.411247,0.00138,0.001426,0.001672
390644,USA,2024-12-31,93132.0,mega,72414.482013,-0.005997,-0.00966,9999.0,1.0,1.0,...,-0.050355,-0.308858,0.645741,0.015584,0.006232,0.558576,4.852698,0.003237,0.003676,0.003112
390645,USA,2024-12-31,93246.0,large,9225.010032,-0.176142,-0.179805,3621.0,1.0,1.0,...,-0.066929,-1.413729,0.353844,0.006286,0.013863,0.452132,3.835753,0.001773,0.002086,0.001655
390646,USA,2024-12-31,93429.0,mega,20455.643761,-0.094742,-0.098405,6211.0,0.0,1.0,...,-0.002226,-0.02675,0.425952,-0.00228,0.00835,0.487186,3.425067,0.002646,0.002843,0.002818


In [8]:
# Save
charc_sp500.to_parquet(datapath / 'JKP_stock_charcs_sp500.parquet', engine='pyarrow')